In [1]:
import pandas as pd

# Load data
excel_file = "Fabric_CU_metrics14days_clean.xlsx"
df = pd.read_excel(excel_file)
csv_file = "fabric_capacity_units.csv"
df.to_csv(csv_file, index=False)

In [2]:
df

,Workspace,Item kind,Item name,CU (s),Duration (s),Users,Rejected count,Billing type
0,MDP-FABRIC-PRD,Dataflow,LEM_DataFlow_PRD,1.267411e+07,347485.003,1,0,Billable
1,MDP-FABRIC-PRD,SynapseNotebook,NB_Infor_101_Ingest_Notebooks,5.341069e+06,105393.327,1,0,Billable
2,MDP-FABRIC-PRD,Dataflow,MasterBuilder_DataFlow_PRD,2.888313e+06,11483.654,1,0,Billable
3,MDP-FABRIC-PRD,SynapseNotebook,NB_Infor_100_Refresh_ReplicationSet_And_GL,1.635235e+06,37332.999,1,0,Billable
4,MDP-FABRIC-PRD,Dataset,Infor_PRD,1.582654e+06,105300.394,3,0,Billable
...,...,...,...,...,...,...,...,...
64,MDP-FABRIC-PRD,Pipeline,PL_FieldForce_100_IngestMaster_AT,2.620800e+02,27031.588,1,0,Billable
65,MDP-FABRIC-PRD,Pipeline,PL_FieldForce_101_IngestOrchestrator_AT,2.620800e+02,26983.733,1,0,Billable
66,MDP-FABRIC-PRD,SynapseNotebook,NB_PipelineStatusEmail,1.391205e+02,34.779,1,0,Billable
67,MDP-FABRIC-PRD,Dataset,Coupa_PRD,2.539200e+01,10.398,3,0,Billable


In [3]:
import pandas as pd


def clean_data(df):
    """Cleans and preprocesses the given DataFrame by:
    - Removing the last row if 'Item name' contains NaN
    - Dropping unnecessary columns
    - Renaming columns for consistency
    - Converting CU(s) to CU(h)

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """

    # Remove last row if 'Item name' is NaN
    if not df.empty and pd.isna(df.iloc[-1]["Item name"]):
        df = df.iloc[:-1]

    # Define columns to drop and rename
    columns_to_drop = {
        "Billing type",
        "Users",
        "Workspace",
        "Item kind",
        "Successful count",
        "Failed count",
        "Duration (s)",
    }

    rename_dict = {
        "Item name": "Item_name",
        "CU (s)": "CU(s)",
        "Duration (s)": "Duration(s)",
    }

    # Drop unnecessary columns (ignore if they don't exist)
    df = df.drop(columns=columns_to_drop, errors="ignore")

    # Rename columns safely
    df = df.rename(columns=rename_dict)

    # Convert CU(s) to CU(h) if CU(s) exists
    if "CU(s)" in df.columns:
        df["CU(h)"] = df["CU(s)"] / 3600

    return df

In [4]:
df = clean_data(df)
df

,Item_name,CU(s),Rejected count,CU(h)
0,LEM_DataFlow_PRD,1.267411e+07,0,3520.587449
1,NB_Infor_101_Ingest_Notebooks,5.341069e+06,0,1483.630283
2,MasterBuilder_DataFlow_PRD,2.888313e+06,0,802.309191
3,NB_Infor_100_Refresh_ReplicationSet_And_GL,1.635235e+06,0,454.231913
4,Infor_PRD,1.582654e+06,0,439.626196
...,...,...,...,...
64,PL_FieldForce_100_IngestMaster_AT,2.620800e+02,0,0.072800
65,PL_FieldForce_101_IngestOrchestrator_AT,2.620800e+02,0,0.072800
66,NB_PipelineStatusEmail,1.391205e+02,0,0.038645
67,Coupa_PRD,2.539200e+01,0,0.007053


### CU Calculation based on Sku Type

In [5]:
def calculate_CU_allocation(sku_capacity, period="day"):
    """Calculates CU allocation in seconds and hours for a given SKU capacity and time period.

    Args:
        sku_capacity (int): The CU capacity of the SKU (e.g., 64 for F64).
        period (str): The time period for calculation. Options: "hour", "day", "month", "year".

    Returns:
        dict: A dictionary with CU-seconds and CU-hours for the specified period.
    """
    # Time conversion factors
    seconds_per_minute = 60
    minutes_per_hour = 60
    hours_per_day = 24
    days_per_month = 30
    days_per_year = 365

    # Define period multipliers
    period_multipliers = {
        "hour": 1 / hours_per_day,  # 1/24 of a day
        "day": 1,
        "month": days_per_month,
        "year": days_per_year,
    }

    if period not in period_multipliers:
        raise ValueError(
            "Invalid period. Choose from 'hour', 'day', 'month', or 'year'."
        )

    # Compute total CU allocation
    multiplier = period_multipliers[period]
    total_CU_seconds = (
        sku_capacity
        * seconds_per_minute
        * minutes_per_hour
        * hours_per_day
        * multiplier
    )
    total_CU_hours = total_CU_seconds / 3600  # Convert to CU-hours

    return {"CU-seconds": total_CU_seconds, "CU-hours": total_CU_hours}

In [6]:
sku_capacity = 64  # Example SKU type (F64)
period = "day"  # Can be "hour", "day", "month", or "year"

cu_allocation = calculate_CU_allocation(sku_capacity, period)

print(
    f"Total CU Allocation for SKU {sku_capacity} ({period}):\n"
    f" - {cu_allocation['CU-seconds']:,} CU-seconds\n"
    f" - {cu_allocation['CU-hours']:,} CU-hours"
)

Total CU Allocation for SKU 64 (day):
 - 5,529,600 CU-seconds
 - 1,536.0 CU-hours


In [7]:
from pyspark.sql.functions import col

# Define global cost rates
TOTAL_DAILYCOST_PAYG = 11.52 * 24
TOTAL_DAILYCOST_RESERVED = 6.853 * 24
TOTAL_CU_HOURS = 1536
WEEK = 7
MONTH = 30
YEAR = 356


def calculate_execution_costs(df_filtered):
    """
    Calculate PAYG and Reserved cost per execution based on Compute Unit (CU) usage.

    Parameters:
    df_filtered (pd.DataFrame): Input DataFrame containing CU(h) column.
    total_CU_hours (int, optional): Total Compute Units per hour for the given capacity. Default is 1536 for F64.

    Returns:
    pd.DataFrame: Updated DataFrame with PAYG and Reserved costs.
    """

    # Calculate cost per execution
    # df_filtered = df_filtered.copy()
    df_filtered["CU(h) per day"] = df_filtered["CU(h)"] / 14
    df_filtered["PAYG Cost($)"] = (
        (df_filtered["CU(h)"] / 14) / TOTAL_CU_HOURS
    ) * TOTAL_DAILYCOST_PAYG
    df_filtered["Reserved Cost($)"] = (
        (df_filtered["CU(h)"] / 14) / TOTAL_CU_HOURS
    ) * TOTAL_DAILYCOST_RESERVED

    print("\n" + "🔹" * 20)
    print("🔹🔹🔹 Cost Per Day for Each Execution 🔹🔹🔹")
    print("🔹" * 20 + "\n")
    return df_filtered


df_filtered = calculate_execution_costs(df)
df_filtered.sort_values(by="CU(h)", ascending=False)


🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
🔹🔹🔹 Cost Per Day for Each Execution 🔹🔹🔹
🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹



,Item_name,CU(s),Rejected count,CU(h),CU(h) per day,PAYG Cost($),Reserved Cost($)
0,LEM_DataFlow_PRD,1.267411e+07,0,3520.587449,251.470532,45.264696,26.926993
1,NB_Infor_101_Ingest_Notebooks,5.341069e+06,0,1483.630283,105.973592,19.075246,11.347453
2,MasterBuilder_DataFlow_PRD,2.888313e+06,0,802.309191,57.307799,10.315404,6.136412
3,NB_Infor_100_Refresh_ReplicationSet_And_GL,1.635235e+06,0,454.231913,32.445137,5.840125,3.474164
4,Infor_PRD,1.582654e+06,0,439.626196,31.401871,5.652337,3.362453
...,...,...,...,...,...,...,...
64,PL_FieldForce_100_IngestMaster_AT,2.620800e+02,0,0.072800,0.005200,0.000936,0.000557
65,PL_FieldForce_101_IngestOrchestrator_AT,2.620800e+02,0,0.072800,0.005200,0.000936,0.000557
66,NB_PipelineStatusEmail,1.391205e+02,0,0.038645,0.002760,0.000497,0.000296
67,Coupa_PRD,2.539200e+01,0,0.007053,0.000504,0.000091,0.000054


In [8]:
# Compute all the costs for all synapse notebooks and pipelines run
# Do this for daily, weekly, monthly, and yearly costs
summary_df = pd.DataFrame(
    {
        "Total CU(h)": [
            df_filtered["CU(h)"].sum(),
            df_filtered["CU(h)"].sum() * WEEK,
            df_filtered["CU(h)"].sum() * MONTH,
            df_filtered["CU(h)"].sum() * YEAR,
        ],
        "Total PAYG Cost ($)": [
            df_filtered["PAYG Cost($)"].sum(),
            df_filtered["PAYG Cost($)"].sum() * WEEK,
            df_filtered["PAYG Cost($)"].sum() * MONTH,
            df_filtered["PAYG Cost($)"].sum() * YEAR,
        ],
        "Total Reserved Cost ($)": [
            df_filtered["Reserved Cost($)"].sum(),
            df_filtered["Reserved Cost($)"].sum() * WEEK,
            df_filtered["Reserved Cost($)"].sum() * MONTH,
            df_filtered["Reserved Cost($)"].sum() * YEAR,
        ],
        "Total CU Utilization(%)": [
            df_filtered["CU(h)"].sum() / TOTAL_CU_HOURS,
            (df_filtered["CU(h)"].sum() * WEEK) / (TOTAL_CU_HOURS * WEEK),
            (df_filtered["CU(h)"].sum() * MONTH) / (TOTAL_CU_HOURS * MONTH),
            (df_filtered["CU(h)"].sum() * YEAR) / (TOTAL_CU_HOURS * YEAR),
        ],
    },
    index=["Daily", "Weekly", "Monthly", "Yearly"],
)

# Print another distinct heading
print("\n" + "🔹" * 30)
print("🔹🔹🔹 COST SUMMARY For All Current Runs Schedules In All Projects 🔹🔹🔹")
print("🔹" * 30 + "\n")

# Display the extended summary DataFrame
display(summary_df)


🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
🔹🔹🔹 COST SUMMARY For All Current Runs Schedules In All Projects 🔹🔹🔹
🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹



,Total CU(h),Total PAYG Cost ($),Total Reserved Cost ($),Total CU Utilization(%)
Daily,9.792186e+03,125.899534,74.894922,6.375121
Weekly,6.854530e+04,881.296737,524.264457,6.375121
Monthly,2.937656e+05,3776.986017,2246.847671,6.375121
Yearly,3.486018e+06,44820.234068,26662.592367,6.375121


In [9]:
(df_filtered["CU(h)"].sum() / 14) * 30 * 0.18

3776.986016999695